In [1]:
import numpy as np
import cupy as cp
import pandas as pd
import time
from typing import Tuple

In [10]:
class GPUSimulatedAnnealingIsing:
    """GPU-accelerated Simulated Annealing for Ising model with fully parallelized operations"""

    def __init__(self, J: cp.ndarray, h: cp.ndarray, n_runs: int = 1024, device_id: int = 0):
        """
        Initialize GPU SA solver with fully vectorized operations
        
        Args:
            J: Coupling matrix (CuPy array)
            h: External field (CuPy array)
            n_runs: Number of parallel SA runs
            device_id: GPU device ID
        """
        cp.cuda.Device(device_id).use()
        self.J = J + J.T
        self.h = h
        self.n = J.shape[0]
        self.n_runs = n_runs

        # Initialize random spin states
        self.states = cp.random.choice([-1, 1], size=(n_runs, self.n)).astype(cp.int8)
        self.best_states = self.states.copy()

        # FIXED: Fully parallelized local field initialization
        self.local_fields = self._initialize_local_fields()
        
        # Calculate initial energies
        self.current_energies = self._calculate_energies(states=self.states)
        self.best_energies = self.current_energies.copy()
        self.global_best_energy = cp.min(self.best_energies)
        self.global_best_state = self.best_states[cp.argmin(self.best_energies)].copy()

    def _initialize_local_fields(self) -> cp.ndarray:
        """
        Parallelized local field initialization (instead of full energy calc)
        L_i = Σ_j J_{ij}s_j + h_i
        
        Returns:
            local_fields: Array of shape (n_runs, n)
        """
        # Vectorized matrix multiplication: (n_runs, n) @ (n, n).T = (n_runs, n)
        # should be faster than the sequential one from before
        local_fields = self.states @ self.J.T + self.h[None, :]
        return local_fields.astype(cp.float32)

    def _calculate_energies(self, states: cp.ndarray) -> cp.ndarray:
        """
        Calculating the Ising model energies for all states
        Energy = Σ_i<j J_ij s_i s_j + Σ_i h_i s_i
        """
        # Interaction term using einsum as its meant to be more efficient
        interaction_energies = 0.5 * cp.einsum('ri,ij,rj->r', states, self.J, states)
        
        # external field term
        field_energies = cp.sum(states * self.h[None, :], axis=1)
        
        return interaction_energies + field_energies

    def solve(self, max_iter: int = 5000, inverse_initial_temp: float = 1.0,
              cooling_rate: float = 0.95, inverse_final_temp: float = 0.01,
              qap_checker=None, check_interval: int = 1) -> Tuple[cp.ndarray, float, float]:
        """
        Run parallel simulated annealing with the optimized energy diff calculations
        
        Args:
            max_iter: Maximum iterations per temperature
            inverse_initial_temp: Initial inverse temperature
            cooling_rate: Temperature cooling rate
            inverse_final_temp: Final inverse temperature
            qap_checker: Optional function to check QAP convergence
            check_interval: Check convergence every N temperature steps
            
        Returns:
            best_state: Ground state configuration
            best_energy: Ground state energy
            time_taken_qap: Time spent on QAP checks
        """
        inverse_temperature = inverse_initial_temp
        iteration = 0
        time_taken_qap_energy = 0
        start_time = time.time()
        temp_step = 0

        while inverse_temperature < inverse_final_temp and iteration < max_iter:
            # sweep through all spins sequentially instead of it being random like before
            for sweep_index in range(self.n):
                
                # Random flip positions for all runs (incase we need to do random)
                # flip_positions = cp.random.randint(0, self.n, size=self.n_runs)

                # Fixed flip postions depending on sweep index
                flip_positions = cp.random.randint(sweep_index, sweep_index + 1, size=self.n_runs)
    
                # Get spins and local fields at flip positions
                run_indices = cp.arange(self.n_runs)
                s_i = self.states[run_indices, flip_positions]
                L_i = self.local_fields[run_indices, flip_positions]
                
                # Calculate delta energy: ΔE_i = -2 * s_i * L_i
                energy_diffs = -2 * s_i * L_i
    
                # Metropolis acceptance
                accept_probs = cp.where(energy_diffs < 0, 1.0, 
                                       cp.exp(-energy_diffs * inverse_temperature))
                accept_mask = cp.random.random(self.n_runs) < accept_probs
    
                # Update local fields for accepted moves
                if cp.any(accept_mask):
                    # Vectorized local field update
                    J_rows = self.J[flip_positions, :]  # (n_runs, n)
                    delta_L = -2 * s_i[:, None] * J_rows  # (n_runs, n)
                    self.local_fields += delta_L * accept_mask[:, None]
                    
                    # Flip the spins (based on the accept masks)
                    self.states[run_indices, flip_positions] *= cp.where(accept_mask, -1, 1)
                    
                    # Update energies based on the energy diffs calculated above
                    self.current_energies[accept_mask] += energy_diffs[accept_mask]
    
                # Update best states
                better_mask = self.current_energies < self.best_energies
                self.best_states[better_mask] = self.states[better_mask]
                self.best_energies[better_mask] = self.current_energies[better_mask]
    
                iteration += 1
                
            inverse_temperature /= cooling_rate
            temp_step += 1
            
            # Update the global best
            current_global_best = cp.min(self.best_energies)
            if current_global_best < self.global_best_energy:
                self.global_best_energy = current_global_best
                self.global_best_state = self.best_states[cp.argmin(self.best_energies)].copy()

            # check the convergence less frequently if needed (setting it as 1 is fine)
            if qap_checker and temp_step % check_interval == 0:
                start_time_qap = time.time()
                current_state_cpu = cp.asnumpy(self.global_best_state)
                
                if qap_checker(current_state_cpu, time.time() - start_time):
                    break
                
                time_taken_qap_energy += time.time() - start_time_qap
        
        end_time = time.time()
        print(f"GPU SA completed in {end_time - start_time:.2f} seconds")

        return self.global_best_state, float(self.global_best_energy), time_taken_qap_energy


class QAPEnergyChecker:
    """Cache QAP matrices instead of constantly performing CSV reads inside the main loop"""
    
    def __init__(self, problem_number: int):
        """Load QAP matrices once at initialization"""
        self.problem_number = problem_number
        self.n = problem_number
        
        # Load matrices once
        F_filepath = f"QAP_J_h/F_tiny0{problem_number}.csv" if problem_number < 10 else f"QAP_J_h/F_tiny{problem_number}.csv"
        D_filepath = f"QAP_J_h/D_tiny0{problem_number}.csv" if problem_number < 10 else f"QAP_J_h/D_tiny{problem_number}.csv"
        
        self.F = pd.read_csv(F_filepath, header=None, skiprows=1).values
        self.D = pd.read_csv(D_filepath, header=None, skiprows=1).values
        
        # Known best energies for convergence checking
        self.known_best = [2.34, 2.21, 5.72, 6.78, 9.35, 9.83, 16.26, 20.27, 22.72, 29.23]
        self.target_energy = self.known_best[problem_number - 3]
        self.converged = False
        self.convergence_time = None
    
    def calculate_energy(self, current_state: np.ndarray) -> float:
        """
        Calculate QAP energy (optimized version without CSV reads)
        This process was converted from Yang's implementation in Matlab
        """
        energy = 0
        spin = current_state.copy()
        spin[spin == -1] = 0
        
        spin2 = spin.reshape(self.n, self.n, order='F')
        
        row_sums = np.sum(spin2, axis=1)
        col_sums = np.sum(spin2, axis=0)
        
        if np.all(row_sums == 1) and np.all(col_sums == 1):
            rows, cols = np.where(spin2 == 1)
            state = np.column_stack([rows + 1, cols + 1])
            
            for i in range(state.shape[0]):
                flow_row = self.F[state[i, 1] - 1, state[:, 1] - 1]
                dist_row = self.D[state[i, 0] - 1, state[:, 0] - 1]
                energy += np.sum(flow_row * dist_row)
        else:
            energy = 100
        
        return energy
    
    def check_convergence(self, current_state: np.ndarray, elapsed_time: float) -> bool:
        """Check if 95% of best known energy is reached"""
        if self.converged:
            return True
        
        current_energy = self.calculate_energy(current_state)
        
        if self.target_energy / current_energy >= 0.95:
            self.convergence_time = elapsed_time
            self.converged = True
            print(f"TT95 = {elapsed_time:.4f} seconds")
            return True
        
        return False


def solve_ising_with_qap_check(J_filepath: str, h_filepath: str, problem_number: int,
                                n_runs: int = 10000, max_iter: int = 5000,
                                inverse_initial_temp: float = 0.5,
                                cooling_rate: float = 0.95,
                                inverse_final_temp: float = 1000.0,
                                check_interval: int = 5) -> dict:
    """
    Solve Ising model with optimized QAP convergence checking
    
    Args:
        check_interval: Check QAP convergence every N temperature steps (reduces overhead but not that important for now)
    """
    # Load matrices
    J = np.loadtxt(J_filepath, delimiter=',', dtype=np.float32)[1:]
    h = np.loadtxt(h_filepath, delimiter=',', dtype=np.float32)[1:]
    
    if h.ndim == 2:
        h = h.flatten()
    
    # Convert to GPU
    J_gpu = cp.asarray(J)
    h_gpu = cp.asarray(h)
    
    # Create QAP checker (loads matrices once)
    qap_checker = QAPEnergyChecker(problem_number)
    
    # Initialize and solve
    gpu_sa = GPUSimulatedAnnealingIsing(J_gpu, h_gpu, n_runs=n_runs)
    
    best_state_gpu, best_energy_gpu, time_qap = gpu_sa.solve(
        max_iter=max_iter,
        inverse_initial_temp=inverse_initial_temp,
        cooling_rate=cooling_rate,
        inverse_final_temp=inverse_final_temp,
        qap_checker=lambda state, t: qap_checker.check_convergence(state, t),
        check_interval=check_interval
    )
    
    best_state_cpu = cp.asnumpy(best_state_gpu)
    final_qap_energy = qap_checker.calculate_energy(best_state_cpu)
    
    return {
        'ground_state_energy': float(best_energy_gpu),
        'qap_energy': final_qap_energy,
        'convergence_time': qap_checker.convergence_time,
        'time_taken_qap_checks': time_qap
    }

In [27]:
# Example usage
if __name__ == "__main__":
    problem_number = 12
    
    J_filepath = f"QAP_J_h/J_tiny0{problem_number}.csv" if problem_number < 10 else f"QAP_J_h/J_tiny{problem_number}.csv"
    h_filepath = f"QAP_J_h/h_tiny0{problem_number}.csv" if problem_number < 10 else f"QAP_J_h/h_tiny{problem_number}.csv"
    
    result = solve_ising_with_qap_check(
        J_filepath, h_filepath, problem_number,
        n_runs=100000,
        max_iter=5000 * problem_number**2,
        inverse_initial_temp=5.0,
        cooling_rate=0.95,
        inverse_final_temp=1000.0,
        check_interval=1  # Only check every 1 temperature steps
    )
    
    print(f"Final QAP Energy: {result['qap_energy']:.2f}")
    print(f"Time to 95%: {result['convergence_time']:.4f}s")

TT95 = 0.6166 seconds
GPU SA completed in 0.62 seconds
Final QAP Energy: 30.58
Time to 95%: 0.6166s
